# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 1:07PM,238780,22,616773,"NBTY Global, Inc.",Released
1,Jun 3 2022 1:06PM,238779,22,616770,"NBTY Global, Inc.",Released
2,Jun 3 2022 12:56PM,238778,102,PBR-052522-02,"Senseonics, Incorporated",Released
3,Jun 3 2022 12:51PM,238777,10,SONSB0001810,"Nextsource Biotechnology, LLC",Released
4,Jun 3 2022 12:50PM,238776,102,PBR-060122-01,"Senseonics, Incorporated",Released
5,Jun 3 2022 12:35PM,238775,10,PRF-28331-BO,Bio-PRF,Released
6,Jun 3 2022 12:35PM,238775,10,PRF-28335-BO,Bio-PRF,Released
7,Jun 3 2022 12:35PM,238775,10,PRF-28486-BO,Bio-PRF,Released
8,Jun 3 2022 12:35PM,238775,10,PRF-29096-BO,Bio-PRF,Released
9,Jun 3 2022 12:35PM,238775,10,PRF-29307-BO,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,238776,Released,1
40,238777,Released,1
41,238778,Released,1
42,238779,Released,1
43,238780,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238776,NaN,NaN,1.0
238777,NaN,NaN,1.0
238778,NaN,NaN,1.0
238779,NaN,NaN,1.0
238780,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238682,11.0,8.0,1.0
238711,0.0,0.0,1.0
238713,0.0,6.0,12.0
238718,1.0,0.0,1.0
238720,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238682,11,8,1
238711,0,0,1
238713,0,6,12
238718,1,0,1
238720,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238682,11,8,1
1,238711,0,0,1
2,238713,0,6,12
3,238718,1,0,1
4,238720,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238682,11,8,1
1,238711,,,1
2,238713,,6,12
3,238718,1,,1
4,238720,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc."
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc."
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated"
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC"
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated"
5,Jun 3 2022 12:35PM,238775,10,Bio-PRF
43,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC"
45,Jun 3 2022 12:09PM,238772,10,Emerginnova
47,Jun 3 2022 12:02PM,238771,12,LF of America Corp.
48,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc.",,,1
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc.",,,1
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated",,,1
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC",,,1
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated",,,1
5,Jun 3 2022 12:35PM,238775,10,Bio-PRF,,,38
6,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",,,2
7,Jun 3 2022 12:09PM,238772,10,Emerginnova,,,2
8,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,,,1
9,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",,3,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc.",1,,
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc.",1,,
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated",1,,
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC",1,,
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated",1,,
5,Jun 3 2022 12:35PM,238775,10,Bio-PRF,38,,
6,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",2,,
7,Jun 3 2022 12:09PM,238772,10,Emerginnova,2,,
8,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,1,,
9,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",32,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc.",1,,
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc.",1,,
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated",1,,
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC",1,,
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc.",1,NaN,NaN
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc.",1,NaN,NaN
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated",1,NaN,NaN
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC",1,NaN,NaN
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 1:07PM,238780,22,"NBTY Global, Inc.",1,0.0,0.0
1,Jun 3 2022 1:06PM,238779,22,"NBTY Global, Inc.",1,0.0,0.0
2,Jun 3 2022 12:56PM,238778,102,"Senseonics, Incorporated",1,0.0,0.0
3,Jun 3 2022 12:51PM,238777,10,"Nextsource Biotechnology, LLC",1,0.0,0.0
4,Jun 3 2022 12:50PM,238776,102,"Senseonics, Incorporated",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4297473,93,24.0,1.0
102,477554,2,0.0,0.0
12,716308,3,0.0,0.0
15,716244,21,6.0,0.0
16,477493,2,0.0,0.0
18,477441,2,0.0,0.0
19,477492,3,0.0,1.0
20,477447,33,11.0,11.0
22,477559,2,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4297473,93,24.0,1.0
1,102,477554,2,0.0,0.0
2,12,716308,3,0.0,0.0
3,15,716244,21,6.0,0.0
4,16,477493,2,0.0,0.0
5,18,477441,2,0.0,0.0
6,19,477492,3,0.0,1.0
7,20,477447,33,11.0,11.0
8,22,477559,2,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93,24.0,1.0
1,102,2,0.0,0.0
2,12,3,0.0,0.0
3,15,21,6.0,0.0
4,16,2,0.0,0.0
5,18,2,0.0,0.0
6,19,3,0.0,1.0
7,20,33,11.0,11.0
8,22,2,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,102,Released,2.0
2,12,Released,3.0
3,15,Released,21.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,22
Status,,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0
Executing,24.0,0.0,0.0,6.0,0.0,0.0,0.0,11.0,0.0
Released,93.0,2.0,3.0,21.0,2.0,2.0,3.0,33.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0
1,Executing,24.0,0.0,0.0,6.0,0.0,0.0,0.0,11.0,0.0
2,Released,93.0,2.0,3.0,21.0,2.0,2.0,3.0,33.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0
1,Executing,24.0,0.0,0.0,6.0,0.0,0.0,0.0,11.0,0.0
2,Released,93.0,2.0,3.0,21.0,2.0,2.0,3.0,33.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()